In [1]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [2]:
import pandas as pd
import numpy as np
import copy
import gensim
import re
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
from spellchecker import SpellChecker

In [3]:
URL_STRING = '__URL_STRING__'

#### Считывание данных

In [4]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-GoogleNews-vectors/GoogleNews-vectors-negative300.bin', binary=True, datatype=np.float16)  


In [5]:
train = pd.read_csv("kaggle_problems/tweet_sentiment_extraction/train.csv")
test = pd.read_csv("kaggle_problems/tweet_sentiment_extraction/test.csv")

#### Samples

In [6]:
train.sample()

,textID,text,selected_text,sentiment
20093,8cab18917e,"I don`t dig emo **** or that, but I like this ...",like,positive


In [7]:
test.sample()

,textID,text,sentiment
2873,34533b5903,FINALLY-- Now I am me again. I had to use int...,neutral


In [8]:
train = train[~train['text'].isnull()]

In [9]:
def paint_urls(s):
    splited = s.split(' ')
    for i in range(len(splited)):
        if splited[i].startswith("http"):
            splited[i] = URL_STRING
    return ' '.join(splited)

def paint_digits(s):
    return re.sub('\d', '#', s)

def preprocessing(data):
    data['text'] = data['text'].apply(paint_urls)
    data['text'] = data['text'].apply(paint_digits)
    
    data['tokens_text'] = data['text'].apply(lambda x: re.split(' |,|\.|\!|\?|\:|\<|\>|\-|\_|\/|\)|\(|\*|\`\'', x.lower()))

In [10]:
preprocessing(train)
preprocessing(test)

In [11]:
train.sample()

,textID,text,selected_text,sentiment,tokens_text
3897,c9807cbece,Psyched for Mother`s day at the GKR party--and...,Psyched for Mother`s day at the GKR party--and...,neutral,"[psyched, for, mother, s, day, at, the, gkr, p..."


In [12]:
words = []
for tokens in train['tokens_text']:
    words.extend(tokens)

In [13]:
words = list(set(words))

In [14]:
print(len(words))

25311


In [19]:
words = words[0:1000]

In [20]:
print(words)

['', 'fml', 'lexi', 'closure', 'charicee', 'archetype', 'feel', 'flavor', "'t", 'yeha', 'dahye', 'operate', 'chilliin', 'via', 'celebxxxvidsyh', 'goooooo', 'effects', 'prolly', "'retail", 'pretties', 'looooooooooooool', 'phasers', 'tables', 'biatches', 'patiently', 'noting', 'btr', 'paige', 'morrrning', 'seal', 'east', 'tweeeet', 'ex', 'montreal', 'cooking', 'cracking', 'memphis', 'dieting', 'teen', 'yeeeeah', 'appts', 'recipe', 'inch', 'dreamt', 'actinggg', 'aces', 'coogars', 'end', 'swine', 'alone', 'blend', 'roxie', 'harmed', 'deposit', 'boulder', 'tirith', 'saddest', 'jamba', 'besos', 'bsame', 'blews', 'jericho', 'biscuits', 'grahmcracker', 'inventor', 'creations', 'blackpool', 'issueï¿½', 'perk', 'sidekick', 'msgs', 'blue', 'fun', 'unfinished', 'claudia', 'bffls', 'diploma', 'tortellini', 'omfg', 'serenading', 'lads', 'americana', 'retrograde', 'cheery', "my'horrible", 'bz', 'pand', 'keyed', 'twitteraddict', 'benefit', 'toilet', 'lia', 'brianï¿½s', 'noah', 'pug', 'blinds', 'declin

In [21]:
%%time
spell = SpellChecker()

cnt_out = 0
cnt_in = 0
for word in words:
    correct_word = spell.correction(word)
    if correct_word != word:
        #print(word, correct_word)
        pass
    try:
        model.get_vector(correct_word)
        cnt_in += 1
    except:
        cnt_out += 1
        print(word, correct_word)

 a
celebxxxvidsyh celebxxxvidsyh
goooooo goo-goo
looooooooooooool looooooooooooool
btr btr
yeeeeah yeeeeah
dreamt dreamt
roxie roxie
tirith tirith
jamba jamba
grahmcracker grahmcracker
issueï¿½ issueï¿½
omfg omf
my'horrible my'horrible
pand and
twitteraddict twitteraddict
brianï¿½s brianï¿½s
'garfields' 'garfields'
wrox wro
amadeus amadeus
[phplurk [phplurk
communifuckingcation communifuckingcation
donï¿½t donï¿½t
pinkpop pinkpop
jiah kiah
homepage' homepage'
#socialmediatv #socialmediatv
lalalaland lalalaland
meyrueis meyrueis
grrrrrr grrrrrr
seeyuhhh seeyuhhh
langley langley
examenul examenul
luckkkk luckkkk
nagrerehab nagrerehab
mazur mamur
pancras pancras
dimitris dimitris
marjorie marjorie
mozert mozart
beshia beshir
quï¿½ quï¿½
sommat sommat
rochelle rochelle
~livelovesing~ ~livelovesing~
inxs inxs
whoishonorsociety whoishonorsociety
carys carys
bradie brodie
brickmans rickman
v=ivfm#pyyucy v=ivfm#pyyucy
houdin houdini
reallyreally reallyreally
ammoxxx ammoxxx
organised organised

In [22]:
print(cnt_out, cnt_in)

159 841


In [206]:
cnt_in_text = defaultdict(int)
cnt_in_selected_text = defaultdict(int)
cnt_throw_away = defaultdict(int)
cnt_throw_away_persent = defaultdict(float)
max_words = 0

In [136]:
# for index, row in train.iterrows():
#     cur_text = list(row['text'].lower().split())
#     for i in cur_text:
#         cnt_in_text[i] += 1
#     max_words = max(max_words, len(cur_text))
    
#     cur_selected_text = list(row['selected_text'].lower().split())
#     for i in cur_selected_text:
#         cnt_in_selected_text[i] += 1
    
# words = set(cnt_in_text.keys()) | set(cnt_in_selected_text.keys())

In [27]:
# print(max_words)

33


In [28]:
# cnt_throw_away = {key: cnt_in_text[key] - cnt_in_selected_text[key] 
#                   for key in words}

# cnt_throw_away_persent = {key: (cnt_in_text[key] - cnt_in_selected_text[key]) / (cnt_in_text[key] + cnt_in_selected_text[key])
#                   for key in words}



In [29]:
# words = set(cnt_in_text.keys()) | set(cnt_in_selected_text.keys())

In [30]:
# sorted(cnt_throw_away_persent.items(), key=lambda item: item[1], reverse=True)

[('zojuist', 1.0),
 ('@_esme_cullen_', 1.0),
 ('jeremy.', 1.0),
 ('menu,', 1.0),
 ('starving!!!', 1.0),
 ('mia!', 1.0),
 ('nail)', 1.0),
 ('uggg', 1.0),
 ('tupac.', 1.0),
 ('munchkins', 1.0),
 ('vacines!', 1.0),
 ('vet,', 1.0),
 ('know...but', 1.0),
 ('overloaded', 1.0),
 ('theory`s', 1.0),
 ('bb,', 1.0),
 ('_cass', 1.0),
 ('pre-application', 1.0),
 ('defrag', 1.0),
 ('sketch', 1.0),
 ('outside...well', 1.0),
 ('19th', 1.0),
 ('a2', 1.0),
 ('1pm...doesn`t', 1.0),
 ('http://tiny12.tv/i3ks8', 1.0),
 ('earphones.', 1.0),
 ('iccvb', 1.0),
 ('begged', 1.0),
 ('ok,thanks.dont', 1.0),
 ('config', 1.0),
 ('dd1.', 1.0),
 ('hell!', 1.0),
 ('http://tinyurl.com/clrbau', 1.0),
 ('hedberg', 1.0),
 ('http://twitpic.com/673vt', 1.0),
 ('http://is.gd/wz2k', 1.0),
 ('sober...dang)!', 1.0),
 ('nero', 1.0),
 ('craving?', 1.0),
 ('bart:', 1.0),
 ('*love*', 1.0),
 ('wade,', 1.0),
 ('walk...', 1.0),
 ('murked!', 1.0),
 ('cares?', 1.0),
 ('it.....and', 1.0),
 ('freakinggg', 1.0),
 ('vacay.', 1.0),
 ('manageab

In [31]:
# sorted(cnt_throw_away_persent.items(), key=lambda item: item[1])

[('sil', -1.0),
 ('.no', -1.0),
 ('pllleeeaaassseeeeee!', -1.0),
 ('bannished.', -1.0),
 ('lonely,d', -1.0),
 ('stomachace...', -1.0),
 ('summe', -1.0),
 ('ruined!!', -1.0),
 ('rocks..', -1.0),
 ('.but', -1.0),
 ('excit', -1.0),
 ('hi..', -1.0),
 ('kudo', -1.0),
 ('.far', -1.0),
 ('fair??', -1.0),
 ('broke..', -1.0),
 ('end..', -1.0),
 ('..loved', -1.0),
 ('faile', -1.0),
 ('president!.', -1.0),
 ('ignor', -1.0),
 ('ruled', -1.0),
 ('pla', -1.0),
 ("w.'", -1.0),
 ('pic..', -1.0),
 ('sadfaced', -1.0),
 ('riot!', -1.0),
 ('ol..i', -1.0),
 ('http://myloc.me/21u', -1.0),
 ('y`s', -1.0),
 ('mommie', -1.0),
 ('.good', -1.0),
 ('ood', -1.0),
 ('http://twitpic.com/664', -1.0),
 ('yayyy!', -1.0),
 ('cried..', -1.0),
 ('innocence!!', -1.0),
 ('goodnight!!', -1.0),
 ('!happy', -1.0),
 ('ug', -1.0),
 ('ing', -1.0),
 ('harde', -1.0),
 ('..definitely', -1.0),
 ('a.wish', -1.0),
 ('breakfast!!', -1.0),
 ('weak!', -1.0),
 ('name..', -1.0),
 ('!thank', -1.0),
 ('books..', -1.0),
 ('awa', -1.0),
 ('norm

#### Проверка гипотезы

In [149]:
#
# Гипотеза: слова из selected_text образуют подотрезок из text
#
cnt_true = 0
cnt_false = 0

for index, row in train.iterrows():
    if row['selected_text'].lower() in row['text'].lower():
        cnt_true += 1
    else:
        cnt_false += 1
print(cnt_true, cnt_false)

27480 0


In [33]:
MAX_WORDS = 35

def selected_text_start(x):
    start_char = x['text'].find(x['selected_text'])
    start_word = len(x['text'][:start_char].split())
    borders = np.zeros(MAX_WORDS, dtype=int)
    borders[start_word] = 1
    return borders

def selected_text_end(x):
    end_word = np.where(x['start_word'] == 1)[0][0] + len(x['selected_text'].split()) - 1
    borders = np.zeros(MAX_WORDS, dtype=int)
    borders[end_word] = 1
    return borders

train['start_word'] = train.apply(lambda x: selected_text_start(x), axis=1)
train['end_word'] = train.apply(lambda x: selected_text_end(x), axis=1)